<a href="https://colab.research.google.com/github/AliArabi55/Dete-net-50/blob/Fine-Tuning/Another_copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -q roboflow transformers torchvision

from roboflow import Roboflow

rf = Roboflow(api_key="XCWKO7J7vBw6fyWimziS")
project = rf.workspace("robouserflow").project("vegetables_v3")
version = project.version(11)
dataset = version.download("coco")


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

##🧩 الجزء 1: تحميل نموذج DETR مع تخصيص عدد الفئات

In [4]:
from transformers import DetrForObjectDetection, DetrImageProcessor

processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

# مجموع الفئات: 91 COCO + 12 فئاتك الجديدة (مع استبعاد "products" وإبقاء "carrot" من COCO نفسها)
num_classes_total = 91 + 11

model = DetrForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50",
    num_labels=num_classes_total,
    ignore_mismatched_sizes=True  # 👈 هذه تضبط التهيئة للتصنيف الجديد
)


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model checkpoin

#🧩 الجزء 2: تعريف أسماء الفئات (COCO + الفئات المخصصة)

In [5]:
# فئات COCO الرسمية
coco_classes = [
    "N/A", "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "street sign", "stop sign", "parking meter", "bench",
    "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe",
    "hat", "backpack", "umbrella", "shoe", "eye glasses", "handbag", "tie", "suitcase",
    "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "plate", "wine glass", "cup", "fork",
    "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot",
    "hot dog", "pizza", "donut", "cake", "chair", "couch", "potted plant", "bed", "mirror",
    "dining table", "window", "desk", "toilet", "door", "tv", "laptop", "mouse", "remote",
    "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "blender",
    "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# فئاتك المخصصة (تم إزالة carrot لأنها رقم 57 في COCO)
custom_classes = [
    "beet", "bell_pepper", "cabbage", "cucumber",
    "egg", "eggplant", "garlic", "onion",
    "potato", "tomato", "zucchini"
]

# قائمة نهائية تضم 102 فئة بالترتيب
id2label = {i: name for i, name in enumerate(coco_classes + custom_classes)}
label2id = {v: k for k, v in id2label.items()}


##✅ الجزء الثالث: إعداد الـ Dataset و DataLoader

loading annotations into memory...
Done (t=0.51s)
creating index...
index created!


In [29]:
from torchvision.datasets import CocoDetection
from torchvision import transforms
from torch.utils.data import DataLoader
import os

# 🧪 إعداد بيانات الاختبار Test
test_image_dir = "/content/Vegetables_v3-11/test"
test_ann_file = os.path.join(test_image_dir, "_annotations.coco.json")

# نفس التحويل المستخدم في التدريب (تحويل إلى Tensor فقط)
test_transform = transforms.Compose([transforms.ToTensor()])

# إعداد Dataset و DataLoader للاختبار
test_dataset = CocoDetection(root=test_image_dir, annFile=test_ann_file, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))


loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [8]:
# عرض جميع الفئات مرتبة حسب ID
for class_id in sorted(id2label.keys()):
    print(f"{class_id:>2} → {id2label[class_id]}")

 0 → N/A
 1 → person
 2 → bicycle
 3 → car
 4 → motorcycle
 5 → airplane
 6 → bus
 7 → train
 8 → truck
 9 → boat
10 → traffic light
11 → fire hydrant
12 → street sign
13 → stop sign
14 → parking meter
15 → bench
16 → bird
17 → cat
18 → dog
19 → horse
20 → sheep
21 → cow
22 → elephant
23 → bear
24 → zebra
25 → giraffe
26 → hat
27 → backpack
28 → umbrella
29 → shoe
30 → eye glasses
31 → handbag
32 → tie
33 → suitcase
34 → frisbee
35 → skis
36 → snowboard
37 → sports ball
38 → kite
39 → baseball bat
40 → baseball glove
41 → skateboard
42 → surfboard
43 → tennis racket
44 → bottle
45 → plate
46 → wine glass
47 → cup
48 → fork
49 → knife
50 → spoon
51 → bowl
52 → banana
53 → apple
54 → sandwich
55 → orange
56 → broccoli
57 → carrot
58 → hot dog
59 → pizza
60 → donut
61 → cake
62 → chair
63 → couch
64 → potted plant
65 → bed
66 → mirror
67 → dining table
68 → window
69 → desk
70 → toilet
71 → door
72 → tv
73 → laptop
74 → mouse
75 → remote
76 → keyboard
77 → cell phone
78 → microwave
79 → o

In [12]:
!pip install -q torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.6/962.6 kB 49.9 MB/s eta 0:00:00


In [19]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torchvision.datasets import CocoDetection
from torchvision import transforms
from transformers import DetrForObjectDetection, DetrImageProcessor
from tqdm import tqdm

# ✅ 1. الإعداد
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_dir = "/content/Vegetables_v3-11/train"
ann_file = f"{image_dir}/_annotations.coco.json"

transform = transforms.Compose([transforms.ToTensor()])
dataset = CocoDetection(root=image_dir, annFile=ann_file, transform=transform)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))



loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [20]:

# ✅ 2. تحميل المعالج والموديل
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", num_labels=102, ignore_mismatched_sizes=True)
model.to(device)

# ✅ 3. التجهيز
optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()

# ✅ 4. تعريف الفئات التي سيتم تتبعها أثناء التدريب
tracked_class_ids = [57, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101]  # carrot + 11 نوع من الداتا الجديدة


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model checkpoin

In [38]:
from transformers import DetrForObjectDetection, DetrImageProcessor
from torch.optim import AdamW
from tqdm import tqdm
import torch
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np

# إعداد الجهاز
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# تحميل النموذج والمعالج
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", ignore_mismatched_sizes=True, num_labels=102)
model.to(device)

# إعداد الأوبتيميزر
optimizer = AdamW(model.parameters(), lr=5e-5)

# الفئات المستهدفة (11 جديدة + carrot)
target_class_ids = list(range(91, 102)) + [57]
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
from tqdm import tqdm

def evaluate_model(model, dataloader, processor, device, target_class_ids, coco_gt_path, tag=""):
    model.eval()
    coco_predictions = []
    image_id = 0

    for images, targets in tqdm(dataloader, desc=f"🔍 Evaluation {tag}"):
        with torch.no_grad():
            pixel_values = processor(images=list(images), return_tensors="pt").pixel_values.to(device)
            outputs = model(pixel_values=pixel_values)

            # ✅ fix: use .size() instead of .size
            target_sizes = [img.size()[::-1] for img in images]
            results = processor.post_process_object_detection(outputs, threshold=0.1, target_sizes=target_sizes)

            for i, result in enumerate(results):
                boxes = result["boxes"].cpu().numpy()
                scores = result["scores"].cpu().numpy()
                labels = result["labels"].cpu().numpy()

                for box, score, label in zip(boxes, scores, labels):
                    if int(label) in target_class_ids:
                        coco_predictions.append({
                            "image_id": int(image_id),
                            "category_id": int(label),
                            "bbox": [
                                float(box[0]),
                                float(box[1]),
                                float(box[2] - box[0]),
                                float(box[3] - box[1])
                            ],
                            "score": float(score)
                        })
                image_id += 1

    # حفظ النتائج مؤقتًا
    results_path = "/tmp/coco_eval_results.json"
    with open(results_path, "w") as f:
        json.dump(coco_predictions, f)

    # تقييم COCO
    coco_gt = COCO(coco_gt_path)
    coco_dt = coco_gt.loadRes(results_path)
    coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')

    # ✅ فلترة الفئات المستهدفة فقط
    coco_eval.params.catIds = target_class_ids

    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    stats = coco_eval.stats
    return {
        "mAP@0.5:0.95": stats[0],
        "mAP@0.5": stats[1],
        "mAP@0.75": stats[2],
        "AR@1": stats[6],
        "AR@10": stats[7],
        "AR@100": stats[8],
    }


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model checkpoin

In [32]:
import os
from glob import glob

# حدد المسارات
base_path = "/content/Vegetables_v3-11"

def count_images(folder):
    return len(glob(os.path.join(folder, "*.jpg"))) + len(glob(os.path.join(folder, "*.png")))

train_count = count_images(os.path.join(base_path, "train"))
valid_count = count_images(os.path.join(base_path, "valid"))
test_count  = count_images(os.path.join(base_path, "test"))

print(f"✅ عدد صور التدريب: {train_count}")
print(f"✅ عدد صور الفاليديشن: {valid_count}")
print(f"✅ عدد صور الاختبار: {test_count}")


✅ عدد صور التدريب: 7787
✅ عدد صور الفاليديشن: 1200
✅ عدد صور الاختبار: 1200


In [36]:
print("📦 عدد الصور في train_loader :", len(train_loader.dataset))
print("📊 عدد الـ batches في train_loader :", len(train_loader))
print("🧮 Batch size المستخدم:", test_loader.batch_size)


📦 عدد الصور في train_loader : 7787
📊 عدد الـ batches في train_loader : 974
🧮 Batch size المستخدم: 16


In [34]:
total_images = sum([len(batch) for batch in test_loader])
print("📷 مجموع الصور:", total_images)


📷 مجموع الصور: 150


In [35]:
for i, (images, _) in enumerate(test_loader):
    print(f"📦 Batch {i} يحتوي على {len(images)} صورة")


📦 Batch 0 يحتوي على 16 صورة
📦 Batch 1 يحتوي على 16 صورة
📦 Batch 2 يحتوي على 16 صورة
📦 Batch 3 يحتوي على 16 صورة
📦 Batch 4 يحتوي على 16 صورة
📦 Batch 5 يحتوي على 16 صورة
📦 Batch 6 يحتوي على 16 صورة
📦 Batch 7 يحتوي على 16 صورة
📦 Batch 8 يحتوي على 16 صورة
📦 Batch 9 يحتوي على 16 صورة
📦 Batch 10 يحتوي على 16 صورة
📦 Batch 11 يحتوي على 16 صورة
📦 Batch 12 يحتوي على 16 صورة
📦 Batch 13 يحتوي على 16 صورة
📦 Batch 14 يحتوي على 16 صورة
📦 Batch 15 يحتوي على 16 صورة
📦 Batch 16 يحتوي على 16 صورة
📦 Batch 17 يحتوي على 16 صورة
📦 Batch 18 يحتوي على 16 صورة
📦 Batch 19 يحتوي على 16 صورة
📦 Batch 20 يحتوي على 16 صورة
📦 Batch 21 يحتوي على 16 صورة
📦 Batch 22 يحتوي على 16 صورة
📦 Batch 23 يحتوي على 16 صورة
📦 Batch 24 يحتوي على 16 صورة
📦 Batch 25 يحتوي على 16 صورة
📦 Batch 26 يحتوي على 16 صورة
📦 Batch 27 يحتوي على 16 صورة
📦 Batch 28 يحتوي على 16 صورة
📦 Batch 29 يحتوي على 16 صورة
📦 Batch 30 يحتوي على 16 صورة
📦 Batch 31 يحتوي على 16 صورة
📦 Batch 32 يحتوي على 16 صورة
📦 Batch 33 يحتوي على 16 صورة
📦 Batch 34 يحتوي على 16 

In [ ]:
# حلقة التدريب + التقييم
best_map = 0.0  # لتتبع أفضل mAP

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    loop = tqdm(train_loader, desc=f"🔥 Training Epoch {epoch}")
    for images, targets in loop:
        pixel_values = processor(images=list(images), return_tensors="pt").pixel_values.to(device)
        target_batch = []

        for target in targets:
            boxes = []
            class_labels = []
            for ann in target:
                if ann["category_id"] in target_class_ids:
                    boxes.append(ann['bbox'])
                    class_labels.append(ann['category_id'])

            if len(boxes) == 0:
                continue

            boxes = torch.tensor(boxes, dtype=torch.float)
            boxes[:, 2:] += boxes[:, :2]  # تحويل من x,y,w,h → x1,y1,x2,y2
            target_batch.append({
                "class_labels": torch.tensor(class_labels, dtype=torch.long).to(device),
                "boxes": boxes.to(device)
            })

        if len(target_batch) == 0:
            continue

        try:
            outputs = model(pixel_values=pixel_values, labels=target_batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loop.set_postfix(loss=loss.item())
        except Exception as e:
            print("❌ Error:", e)

    # التقييم في نهاية كل Epoch
    metrics = evaluate_model(
        model=model,
        dataloader=test_loader,
        processor=processor,
        device=device,
        target_class_ids=target_class_ids,
        coco_gt_path="/content/Vegetables_v3-11/test/_annotations.coco.json",
        tag=f"Epoch {epoch}"
    )
    print(f"📊 نتائج التقييم بعد Epoch {epoch}: {metrics}")

    # ✅ حفظ النموذج الأفضل بناءً على mAP@0.5:0.95
    if metrics["mAP@0.5:0.95"] > best_map:
        best_map = metrics["mAP@0.5:0.95"]
        torch.save(model.state_dict(), "best_detr_model.pth")
        print(f"💾 تم حفظ النموذج الأفضل عند Epoch {epoch} بـ mAP@0.5:0.95 = {best_map:.4f}")


🔥 Training Epoch 0:  11%|█         | 109/974 [01:04<08:11,  1.76it/s]

In [46]:
import os

json_dir = "/content/Vegetables_v3-11/test"

# التأكد من وجود المجلد
if os.path.exists(json_dir):
    # الحصول على جميع الملفات .json مع مساراتها الكاملة
    json_files = [
        os.path.join(json_dir, f)
        for f in os.listdir(json_dir)
        if f.endswith(".json")
    ]

    # طباعة النتائج
    if json_files:
        print("📄 ملفات JSON الموجودة (المسارات الكاملة):")
        for file in json_files:
            print(file)
    else:
        print("❌ لا توجد ملفات JSON في هذا المسار.")
else:
    print(f"❌ المسار غير موجود: {json_dir}")


📄 ملفات JSON الموجودة (المسارات الكاملة):
/content/Vegetables_v3-11/test/_annotations.coco.json


In [23]:

# 📊 مثال لربط دالة التقييم مع التدريب:
for epoch in range(num_epochs):
    model.train()
    loop = tqdm(train_loader, leave=True, desc=f"🛠️ Training Epoch {epoch}")

    for images, targets in loop:
        pixel_values = processor(images=list(images), return_tensors="pt").pixel_values.to(device)

        target_batch = []
        for target in targets:
            boxes = []
            class_labels = []

            for ann in target:
                if ann["category_id"] in target_category_ids:
                    boxes.append(ann['bbox'])
                    class_labels.append(ann['category_id'])

            if len(boxes) == 0:
                continue

            boxes = torch.tensor(boxes, dtype=torch.float)
            boxes[:, 2:] += boxes[:, :2]  # x,y,w,h → x1,y1,x2,y2

            target_batch.append({
                "class_labels": torch.tensor(class_labels, dtype=torch.long).to(device),
                "boxes": boxes.to(device)
            })

        if len(target_batch) == 0:
            continue

        try:
            outputs = model(pixel_values=pixel_values, labels=target_batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loop.set_postfix(loss=loss.item())
        except Exception as e:
            print("❌ Error:", e)

    # 📊 التقييم بعد كل Epoch
    print(f"📈 Evaluation after Epoch {epoch}")
    evaluate_model(
        model=model,
        dataloader=val_loader,
        processor=processor,
        device=device,
        valid_category_ids=target_category_ids,
        epoch_num=epoch
    )
    display_eval_table()

# 🏁 التقييم النهائي
print("🏑 Final Evaluation After All Epochs")
evaluate_model(
    model=model,
    dataloader=val_loader,
    processor=processor,
    device=device,
    valid_category_ids=target_category_ids,
    epoch_num="Final"
)
display_eval_table()

NameError: name 'num_epochs' is not defined

In [21]:
# ✅ 5. التدريب
for epoch in range(5):
    loop = tqdm(train_loader, leave=True, desc=f"Training Epoch {epoch}")
    for images, targets in loop:
        # تجهيز الصور
        pixel_values = processor(images=list(images), return_tensors="pt").pixel_values.to(device)

        # تجهيز الأهداف (boxes و class_labels)
        target_batch = []
        for target in targets:
            boxes = []
            class_labels = []

            for ann in target:
                if ann["category_id"] in tracked_class_ids:
                    boxes.append(ann['bbox'])
                    class_labels.append(ann['category_id'])

            if len(boxes) == 0:
                continue  # تجاوز إن لم يكن هناك فئات متعقبة

            boxes = torch.tensor(boxes, dtype=torch.float)
            boxes[:, 2:] += boxes[:, :2]  # تحويل من (x, y, w, h) إلى (x1, y1, x2, y2)

            target_batch.append({
                "class_labels": torch.tensor(class_labels, dtype=torch.long).to(device),
                "boxes": boxes.to(device)
            })

        # تجاوز في حال لم يكن هناك batch مفيد
        if len(target_batch) == 0:
            continue

        try:
            outputs = model(pixel_values=pixel_values, labels=target_batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loop.set_postfix(loss=loss.item())
        except Exception as e:
            print("❌ Error:", e)

Training Epoch 1:  81%|████████  | 789/974 [07:53<01:51,  1.66it/s]


KeyboardInterrupt: 

In [15]:
import torch
import pandas as pd
from tqdm import tqdm
import torchmetrics
from transformers import DetrImageProcessor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

# ✅ فئات التقييم: carrot من COCO = 57، والباقي من 91 إلى 101
valid_class_ids = [57] + list(range(91, 102))
evaluation_log = []

In [16]:


# 🧱 الجزء 1: دالة التقييم بعد كل Epoch
def evaluate_model(model, processor, val_loader, epoch_num):
    model.eval()
    metric_map = torchmetrics.detection.MeanAveragePrecision(iou_type="bbox", class_metrics=True).to(device)

    with torch.no_grad():
        for images, targets in tqdm(val_loader, desc=f"Evaluating Epoch {epoch_num}"):
            pixel_values = processor(images=list(images), return_tensors="pt").pixel_values.to(device)

            outputs = model(pixel_values=pixel_values)
            pred_logits = outputs.logits.softmax(-1)
            pred_scores, pred_labels = pred_logits.max(-1)

            results = []
            for i in range(len(images)):
                boxes = outputs.pred_boxes[i].cpu()
                scores = pred_scores[i].cpu()
                labels = pred_labels[i].cpu()

                results.append({
                    "boxes": boxes,
                    "scores": scores,
                    "labels": labels
                })

            ground_truth = []
            for t in targets:
                gt_labels = torch.tensor([ann["category_id"] for ann in t])
                gt_boxes = torch.tensor([ann["bbox"] for ann in t], dtype=torch.float)
                if gt_boxes.ndim == 2:
                    gt_boxes[:, 2:] += gt_boxes[:, :2]  # x,y,w,h → x_min,y_min,x_max,y_max
                ground_truth.append({
                    "labels": gt_labels,
                    "boxes": gt_boxes
                })

            metric_map.update(results, ground_truth)

    final_result = metric_map.compute()
    summary_row = {
        "Epoch": epoch_num,
        "mAP@0.5:0.95": final_result["map"].item(),
        "mAP@0.5": final_result["map_50"].item(),
        "mAP@0.75": final_result["map_75"].item(),
        "AR@1": final_result["mar_1"].item(),
        "AR@10": final_result["mar_10"].item(),
        "AR@100": final_result["mar_100"].item()
    }

    for cls_id in valid_class_ids:
        class_str = f"Class_{cls_id}"
        precision = final_result["class_precision"].get(cls_id, torch.tensor(0.0)).item()
        recall = final_result["class_recall"].get(cls_id, torch.tensor(0.0)).item()
        f1 = 2 * precision * recall / (precision + recall + 1e-6)

        summary_row[f"{class_str}_P"] = precision
        summary_row[f"{class_str}_R"] = recall
        summary_row[f"{class_str}_F1"] = f1

    evaluation_log.append(summary_row)
    model.train()

In [17]:

# 🧱 الجزء 2: عرض الجدول وتصديره

def display_eval_table():
    df = pd.DataFrame(evaluation_log)
    display(df)
    df.to_csv("/content/eval_progress.csv", index=False)

In [18]:

# 🧱 الجزء 3: كود التدريب المحدث

for epoch in range(5):
    loop = tqdm(train_loader, desc=f"Training Epoch {epoch}")
    for images, targets in loop:
        pixel_values = processor(images=list(images), return_tensors="pt").pixel_values.to(device)

        # تجهيز الأنوتيشنات
        target_batch = []
        for target in targets:
            boxes = [ann["bbox"] for ann in target]
            class_labels = [ann["category_id"] for ann in target]

            boxes = torch.tensor(boxes, dtype=torch.float)
            if boxes.ndim == 2:
                boxes[:, 2:] += boxes[:, :2]  # x,y,w,h → x_min,y_min,x_max,y_max

            target_batch.append({
                "class_labels": torch.tensor(class_labels, dtype=torch.long),
                "boxes": boxes
            })

        try:
            outputs = model(pixel_values=pixel_values, labels=target_batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loop.set_postfix(loss=loss.item())
        except Exception as e:
            print("❌ Error:", e)

Training Epoch 0:   0%|          | 1/487 [00:02<22:26,  2.77s/it]

❌ Error: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument x2 in method wrapper_CUDA___cdist_forward)


Training Epoch 0:   0%|          | 2/487 [00:04<18:30,  2.29s/it]

❌ Error: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument x2 in method wrapper_CUDA___cdist_forward)


Training Epoch 0:   1%|          | 3/487 [00:06<17:00,  2.11s/it]

...

❌ Error: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument x2 in method wrapper_CUDA___cdist_forward)


Training Epoch 0:  81%|████████▏ | 396/487 [13:05<02:57,  1.95s/it]

❌ Error: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument x2 in method wrapper_CUDA___cdist_forward)


Training Epoch 0:  81%|████████▏ | 396/487 [13:06<03:00,  1.99s/it]


KeyboardInterrupt: 

In [14]:




    # ✅ بعد كل Epoch: التقييم والعرض
    evaluate_model(model, processor, val_loader, epoch_num=epoch)
    display_eval_table()


Training Epoch 0:   0%|          | 1/487 [00:03<27:36,  3.41s/it]

❌ Error: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument x2 in method wrapper_CUDA___cdist_forward)


Training Epoch 0:   0%|          | 2/487 [00:05<22:59,  2.84s/it]

❌ Error: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument x2 in method wrapper_CUDA___cdist_forward)


Training Epoch 0:   1%|          | 3/487 [00:08<22:03,  2.74s/it]

...

❌ Error: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument x2 in method wrapper_CUDA___cdist_forward)


Training Epoch 0:   2%|▏         | 8/487 [00:18<16:59,  2.13s/it]

❌ Error: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument x2 in method wrapper_CUDA___cdist_forward)


Training Epoch 0:   2%|▏         | 8/487 [00:19<19:35,  2.45s/it]


KeyboardInterrupt: 

##✅ 2. تحميل البيانات كـ COCO Dataset

In [ ]:
from torchvision.datasets import CocoDetection
from torchvision import transforms

image_dir = "/content/Vegetables_v3-11/train"
ann_file = "/content/Vegetables_v3-11/train/_annotations.coco.json"

transform = transforms.Compose([transforms.ToTensor()])
train_dataset = CocoDetection(root=image_dir, annFile=ann_file, transform=transform)


loading annotations into memory...
Done (t=0.83s)
creating index...
index created!


##✅ 3. DataLoader

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)


##✅ 4. تحميل DETR والمعالج

In [ ]:
import json

coco_path = "/content/Vegetables_v3-11/train/_annotations.coco.json"

with open(coco_path, 'r') as f:
    coco_json = json.load(f)

num_classes = len(coco_json['categories'])
print("✅ Number of classes:", num_classes)


✅ Number of classes: 13


In [ ]:
from transformers import DetrForObjectDetection, DetrImageProcessor

processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

model = DetrForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50",
    num_labels=num_classes,
    ignore_mismatched_sizes=True  # ← مهم جدًا لتفادي خطأ حجم التصنيفات
)



The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
for bn1.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
for layer1.0.conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)


In [ ]:
import torch
from torch.optim import AdamW
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()


DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

##✅ أولًا: استخراج معلومات عن مجموعة البيانات (Data Info)

#📊 عدد الصور في كل جزء:

In [ ]:
import os
from glob import glob

# حدد المسارات
base_path = "/content/Vegetables_v3-11"

def count_images(folder):
    return len(glob(os.path.join(folder, "*.jpg"))) + len(glob(os.path.join(folder, "*.png")))

train_count = count_images(os.path.join(base_path, "train"))
valid_count = count_images(os.path.join(base_path, "valid"))
test_count  = count_images(os.path.join(base_path, "test"))

print(f"✅ عدد صور التدريب: {train_count}")
print(f"✅ عدد صور الفاليديشن: {valid_count}")
print(f"✅ عدد صور الاختبار: {test_count}")


✅ عدد صور التدريب: 7787
✅ عدد صور الفاليديشن: 1200
✅ عدد صور الاختبار: 1200


##🏷️ عدد الكلاسات وأسماؤها:

In [ ]:
import json

with open(f"{base_path}/train/_annotations.coco.json") as f:
    coco_data = json.load(f)

categories = coco_data['categories']
class_names = [cat['name'] for cat in categories]

print(f"✅ عدد الفئات: {len(class_names)}")
print(f"📌 أسماء الفئات: {class_names}")


✅ عدد الفئات: 13
📌 أسماء الفئات: ['products', 'beet', 'bell_pepper', 'cabbage', 'carrot', 'cucumber', 'egg', 'eggplant', 'garlic', 'onion', 'potato', 'tomato', 'zucchini']


##✅ الجزء 1 – تحميل البيانات والتحويلات



In [ ]:
from torchvision.datasets import CocoDetection
from torchvision import transforms

image_dir = "/content/Vegetables_v3-11/train"
ann_file = "/content/Vegetables_v3-11/train/_annotations.coco.json"

transform = transforms.Compose([transforms.ToTensor()])
train_dataset = CocoDetection(root=image_dir, annFile=ann_file, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))


loading annotations into memory...
Done (t=0.11s)
creating index...
index created!


In [ ]:
IGNORED_CLASSES = ['products']
class_name_to_id = {cat['name']: cat['id'] for cat in categories if cat['name'] not in IGNORED_CLASSES}
valid_class_ids = set(class_name_to_id.values())


In [ ]:
# الكلاسات النهائية (باستثناء 'products')
class_names = ['beet', 'bell_pepper', 'cabbage', 'carrot', 'cucumber',
               'egg', 'eggplant', 'garlic', 'onion', 'potato', 'tomato', 'zucchini']

# افترض أن IDs تبدأ من 0 إلى 11 بترتيب class_names
class_name_to_id = {name: i for i, name in enumerate(class_names)}
valid_class_ids = set(class_name_to_id.values())  # IDs المسموح بها


##✅ ثانيًا: تعديل تجهيز الـtargets داخل الـfor loop في كود التدريب

In [ ]:
for images, targets in loop:
    ...
    # ⬇️ هذا داخل الحلقة وليس خارجها
    target_batch = []
    for target in targets:
        boxes = []
        class_labels = []
        for ann in target:
            if ann['category_id'] not in valid_class_ids:
                continue
            boxes.append(ann['bbox'])
            class_labels.append(ann['category_id'])

        if boxes:
            boxes = torch.tensor(boxes, dtype=torch.float)
            boxes[:, 2:] += boxes[:, :2]
            target_batch.append({
                "class_labels": torch.tensor(class_labels, dtype=torch.long).to(device),
                "boxes": boxes.to(device)
            })


NameError: name 'loop' is not defined

In [ ]:
for epoch in range(num_epochs):
    loop = tqdm(train_loader, leave=True)
    for images, targets in loop:
        pixel_values = processor(images=list(images), return_tensors="pt").pixel_values.to(device)

        # تجهيز الأنوتيشنات بالتنسيق المناسب لـ DETR
        target_batch = []
        for target in targets:
            boxes = []
            class_labels = []
            for ann in target:
                if ann['category_id'] not in valid_class_ids:
                    continue
                boxes.append(ann['bbox'])
                class_labels.append(ann['category_id'])

            if boxes:
                boxes = torch.tensor(boxes, dtype=torch.float)
                boxes[:, 2:] += boxes[:, :2]  # (x,y,w,h) → (x1,y1,x2,y2)
                target_batch.append({
                    "class_labels": torch.tensor(class_labels, dtype=torch.long).to(device),
                    "boxes": boxes.to(device)
                })

        try:
            outputs = model(pixel_values=pixel_values, labels=target_batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loop.set_description(f"Epoch {epoch}")
            loop.set_postfix(loss=loss.item())
        except Exception as e:
            print("❌ Error:", e)


NameError: name 'targets' is not defined

In [ ]:
for ann in target:
    if ann['category_id'] not in valid_class_ids:
        continue
    boxes.append(ann['bbox'])
    class_labels.append(ann['category_id'])


NameError: name 'target' is not defined

In [ ]:
import torch
from torch.optim import AdamW
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()

for epoch in range(10):  # يمكنك تعديل عدد الـ epochs حسب الحاجة
    loop = tqdm(train_loader, leave=True)
    for images, targets in loop:
        try:
            # تأكد من أن كل الصور على الجهاز الصحيح
            pixel_values = processor(images=list(images), return_tensors="pt", do_rescale=False).pixel_values.to(device)

            # تجهيز الأنوتيشنات بصيغة DETR
            target_batch = []
            for target in targets:
                boxes = []
                class_labels = []
                for ann in target:
                    boxes.append(ann['bbox'])  # (x, y, w, h)
                    class_labels.append(ann['category_id'])

                boxes = torch.tensor(boxes, dtype=torch.float)

                # تأكد أن boxes في شكل [N, 4]
                if boxes.ndim == 1:
                    boxes = boxes.unsqueeze(0)

                # تحويل (x, y, w, h) → (x_min, y_min, x_max, y_max)
                boxes[:, 2:] += boxes[:, :2]

                target_batch.append({
                    "class_labels": torch.tensor(class_labels, dtype=torch.long).to(device),
                    "boxes": boxes.to(device)
                })

            # التمرير الأمامي والحساب
            outputs = model(pixel_values=pixel_values, labels=target_batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loop.set_description(f"Epoch {epoch}")
            loop.set_postfix(loss=loss.item())

        except Exception as e:
            print("❌ Error:", e)


Epoch 0:   1%|          | 4/487 [00:10<19:57,  2.48s/it, loss=6.04e+3]

❌ Error: Sizes of tensors must match except in dimension 0. Expected size 4 but got size 0 for tensor number 3 in the list.


Epoch 0:   1%|          | 5/487 [00:12<17:46,  2.21s/it, loss=6.04e+3]

❌ Error: Sizes of tensors must match except in dimension 0. Expected size 4 but got size 0 for tensor number 12 in the list.


Epoch 0:   2%|▏         | 9/487 [00:22<18:43,  2.35s/it, loss=5.86e+3]

...

Epoch 0:  63%|██████▎   | 309/487 [12:59<06:45,  2.28s/it, loss=6.55e+3]

❌ Error: Sizes of tensors must match except in dimension 0. Expected size 0 but got size 4 for tensor number 1 in the list.


Epoch 0:  65%|██████▍   | 315/487 [13:15<07:10,  2.50s/it, loss=6.52e+3]

❌ Error: Sizes of tensors must match except in dimension 0. Expected size 4 but got size 0 for tensor number 2 in the list.


Epoch 0:  65%|██████▌   | 318/487 [13:25<07:30,  2.67s/it, loss=6.99e+3]

##✅ 5. Fine-Tuning Training Loop


In [ ]:
import torch
from torch.optim import AdamW
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()

for epoch in range(10):  # عدل حسب رغبتك
    loop = tqdm(train_loader, leave=True)
    for images, targets in loop:
        pixel_values = processor(images=list(images), return_tensors="pt").pixel_values.to(device)

        # تجهيز التنسيقات المطلوبة لـ DETR
        target_batch = []
        for target in targets:
            boxes = []
            class_labels = []
            for ann in target:
                boxes.append(ann['bbox'])
                class_labels.append(ann['category_id'])

            # تحويل (x, y, w, h) → (x_min, y_min, x_max, y_max)
            boxes = torch.tensor(boxes, dtype=torch.float)
            boxes[:, 2:] += boxes[:, :2]

            target_batch.append({
                "class_labels": torch.tensor(class_labels, dtype=torch.long),
                "boxes": boxes
            })

        try:
            outputs = model(pixel_values=pixel_values, labels=target_batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loop.set_description(f"Epoch {epoch}")
            loop.set_postfix(loss=loss.item())
        except Exception as e:
            print("❌ Error:", e)


##✅ 6. حفظ النموذج

In [ ]:
model.save_pretrained("/content/best_detr_model")
processor.save_pretrained("/content/best_detr_model")